# Calculate Pollutant Loading from Earth Engine

This notebook demonstrates an example workflow for calculating zonal statistics for runoff and pollutant loading. 

---

## Import libaries and authenticate to Earth Engine

In [ ]:
import json
import ee

service_account = "tnc-data-v1@appspot.gserviceaccount.com"
credentials = ee.ServiceAccountCredentials(
    service_account, "/Users/christiannilsen/Documents/secrets/tncKey.json"
)  ## replace with local file or env
ee.Initialize(credentials)

## Get the Runoff and Concentration Rasters 

In [ ]:
runoff = ee.Image(
    "projects/ee-stormwaterheatmap/assets/production/Mean_Annual_Q_4_epochs"
)
concentrations = ee.Image(
    "projects/ee-stormwaterheatmap/assets/production/coc_concentrations"
)

## View image information 

Image properties can be accessed by casting the image to a dictionary. It will return just the properties as a JSON. 

Note the ```getInfo()``` function is needed to pull data from Earth Engine to the client. 

In [ ]:
q_dict = runoff.toDictionary()
c_dict = concentrations.toDictionary()

### Runoff Image Properties

In [ ]:
print(json.dumps(q_dict.getInfo(), indent=4))

### Concentration Image Properties

In [ ]:
print(json.dumps(c_dict.getInfo(), indent=4))

## Calculate a loading layer

In [ ]:
# What bands are available from the runoff image?
(runoff.bandNames().getInfo())

In [ ]:
# Select the first epoch
Q = runoff.select(0)

# make a multiband image with unit pollutant loading
unit_loads = Q.multiply(concentrations).multiply(
    ee.Image(1e-9)
)  # units for this image are kg/m2/year

# Use ee.PixelArea() to get the area for each pixel. This image will be in m2. Multiply it by the unit load image
loads = unit_loads.multiply(ee.Image.pixelArea())

# Use the layer names to rename the bands to make it pretty
layer_names = c_dict.get("layer_names").getInfo().split(", ")
loads = loads.rename(layer_names)

In [ ]:
# View layer names
layer_names

### Display image of loading 

In [ ]:
# study area bounds
Tacoma = (
    ee.FeatureCollection("projects/ee-tacoma-watershed/assets/vector/study_area")
    .union()
    .geometry()
)

In [ ]:
from IPython.display import Image

# Display a thumbnail of unit loads just to see.
Image(
    url=loads.select("Total Suspended Solids")
    .clip(Tacoma)
    .getThumbUrl(
        {"min": 0, "max": 100, "dimensions": 512, "palette": ["#e4f1e1", "#7c1d6f"]}
    )
)

## Calculate Zonal Stats

In [ ]:
# pull in some vectors
subwatersheds = ee.FeatureCollection(
    "projects/ee-tacoma-watershed/assets/vector/Tacoma_SWPolys_swSubbasin"
)

# calculate load per subwatershed
load_stats = loads.reduceRegions(
    collection=subwatersheds, reducer=ee.Reducer.sum(), scale=5
)  # sum of values for each feature. For mean values, use ee.Reducer.mean()

Zonal stats as dataframe:  

In [ ]:
import pandas as pd

# pull from server
load_stats_dict = load_stats.getInfo()

# get data as a pandas dataframe
load_stats_df = pd.DataFrame()
for feat in load_stats_dict["features"]:
    df = pd.DataFrame([feat["properties"]], columns=feat["properties"].keys())
    load_stats_df = pd.concat([load_stats_df, df], sort=True, axis=0)

load_stats_df

Zonal stats as geodataframe

In [ ]:
# as a geopandas df
import geopandas as gpd

gdf = gpd.GeoDataFrame.from_features(load_stats_dict.get("features"))
gdf

In [ ]:
# Plot geodataframe
gdf.plot(column="Total Suspended Solids")